## Building a Recommendation engine in Julia


### What is Julia : 

Julia is a **high-level, high-performance dynamic programming language for technical computing**, with syntax that is familiar to users of other technical computing environments. It provides a sophisticated compiler, distributed parallel execution, numerical accuracy, and an extensive mathematical function library.



### Recommendation System

#### Examples

* Netflix
* Amazon
* Facebook
* Last.fm

#### What are they?

* A system which understands the users personal taste for various products.
* The products can range from items on e-commerce sites to movies to connections on social sites.
* It uses the users past behaviour when available, like previous ratings on movies watched previously etc.
* Mathematically models the user-item interaction using complex algorithms.
* Works on large sparse datasets, which consists of the past behaviour.
* Make high quality predictions to individual users.

P.S : Certain limitations do exist, like lack of sufficient information of the users and movies. We assume that each user must rate a certain minimum number of movies in order to get accurate predictions.


### What it takes to build one :

 1. ** A versatile programming language **.
 2. ** Powerful mathematical capabilities **.
 3. High-performance, parallelism, ability to scale.
 4. Web, javascript capabilities.
 
### Why Julia :

1. Its syntax which is similar Python and Matlab. 
2. Julia runs inherently fast, written well.
3. Easy to call C/Fortran.
4. Ready for handling Big Data.
5. Add more.. 

### Toy Model of the movie recommender system :

Consider 4 users *Alice*, *Bob*, *John*, *Dow* and 4 movies *Titanic*, *Braveheart*, *Lion King*, *Troll 2*.

|   |Titanic   |Braveheart   |Lion King   | Troll 2  |
|---|:---:|:---:|:---:|:---:|
|Alice| 4  | 5  | 1  | ?  |
| Bob  | 5  | ?  | 2  | 1  |
| John  | 2  | ?  | 4  | 4  |
| Dow  | ?  | 2  | 2  | 4  |

The table above shows the ratings given by the 4 users to the 4 movies. The `?` are the movies unseen and hence we need to predict the ratings. By setting a threshold, say 3.5, we can recommend movies whose predictions are greater than 3.5.

In [1]:
# Rating martix, R

R = [4 5 1 0;
     5 0 2 1;
     2 0 4 4;
     0 2 2 5;]

4×4 Array{Int64,2}:
 4  5  1  0
 5  0  2  1
 2  0  4  4
 0  2  2  5

#### Low rank matrix approximations to predict the movie ratings



In [2]:
# First let us filter out the unwatched movie ratings.
R_s = sparse(R)

4×4 sparse matrix with 12 Int64 nonzero entries:
	[1, 1]  =  4
	[2, 1]  =  5
	[3, 1]  =  2
	[1, 2]  =  5
	[4, 2]  =  2
	[1, 3]  =  1
	[2, 3]  =  2
	[3, 3]  =  4
	[4, 3]  =  2
	[2, 4]  =  1
	[3, 4]  =  4
	[4, 4]  =  5

In [3]:
?svds

search: svds svdvals svdvals! svd svdfact svdfact! isvalid



```
svds(A; nsv=6, ritzvec=true, tol=0.0, maxiter=1000, ncv=2*nsv, u0=zeros((0,)), v0=zeros((0,))) -> (SVD([left_sv,] s, [right_sv,]), nconv, niter, nmult, resid)
```

Computes the largest singular values `s` of `A` using implicitly restarted Lanczos iterations derived from [`eigs`](:func:`eigs`).

**Inputs**

  * `A`: Linear operator whose singular values are desired. `A` may be represented as a subtype of `AbstractArray`, e.g., a sparse matrix, or any other type supporting the four methods `size(A)`, `eltype(A)`, `A * vector`, and `A' * vector`.
  * `nsv`: Number of singular values. Default: 6.
  * `ritzvec`: If `true`, return the left and right singular vectors `left_sv` and `right_sv`.  If `false`, omit the singular vectors. Default: `true`.
  * `tol`: tolerance, see [`eigs`](:func:`eigs`).
  * `maxiter`: Maximum number of iterations, see [`eigs`](:func:`eigs`). Default: 1000.
  * `ncv`: Maximum size of the Krylov subspace, see [`eigs`](:func:`eigs`) (there called `nev`). Default: `2*nsv`.
  * `u0`: Initial guess for the first left Krylov vector. It may have length `m` (the first dimension of `A`), or 0.
  * `v0`: Initial guess for the first right Krylov vector. It may have length `n` (the second dimension of `A`), or 0.

**Outputs**

  * `svd`: An `SVD` object containing the left singular vectors, the requested values, and the right singular vectors. If `ritzvec = false`, the left and right singular vectors will be empty.
  * `nconv`: Number of converged singular values.
  * `niter`: Number of iterations.
  * `nmult`: Number of matrix–vector products used.
  * `resid`: Final residual vector.

**Example**

```julia
X = sprand(10, 5, 0.2)
svds(X, nsv = 2)
```

**Implementation note**

`svds(A)` is formally equivalent to calling `eigs` to perform implicitly restarted Lanczos tridiagonalization on the Hermitian matrix $\begin{pmatrix} 0 & A^\prime \\ A & 0 \end{pmatrix}$, whose eigenvalues are plus and minus the singular values of $A$.


In [4]:
# Let `k=3` be the reduced rank
k=3
LR = svds(R_s, nsv=k)

(Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}([-0.49182 0.695271 -0.469264; -0.470859 0.284387 0.660791; -0.562667 -0.424538 0.284075; -0.468841 -0.50546 -0.512297],[9.3931,5.91163,4.10572],[-0.579884 0.567346 0.48592; -0.361625 0.417048 -0.821029; -0.492051 -0.244438 0.2348; -0.539303 -0.666663 -0.186178]),6,1,8,[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [5]:
# Let us reconstruct back the matrix
U = LR[1][:U]
S = LR[1][:S]
V = LR[1][:Vt]

4×3 Array{Float64,2}:
 -0.579884   0.567346   0.48592 
 -0.361625   0.417048  -0.821029
 -0.492051  -0.244438   0.2348  
 -0.539303  -0.666663  -0.186178

In [6]:
R_new = convert(Array{Int64}, ceil(U*(diagm(S)*V')))

4×4 Array{Int64,2}:
 5  5  1  1
 5  1  3  1
 3  0  4  5
 0  3  3  5

In [7]:
R

4×4 Array{Int64,2}:
 4  5  1  0
 5  0  2  1
 2  0  4  4
 0  2  2  5

### Exploratory analysis and plots of the data

The data is sourced from [GroupLens](http://grouplens.org/datasets/movielens/), we choose the 20 million ratings dataset, which is obtained from 138,000 users who have rated 27,000 movies.

** Reference : **
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872

In [8]:
#path = joinpath("Users","abhijith","work","ML","notebooks","data","recommender","ml-20m")
#ratings_file = "ratings.csv"
#df_ratings = readtable(joinpath(path, ratings_file))

if !isdir(Pkg.dir("RecSys"))  Pkg.add("DistributedArrays");   end
using DataFrames
df_ratings = readtable("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/ratings.csv");
head(df_ratings)

,userId,movieId,rating,timestamp
1,1,2,3.5,1112486027
2,1,29,3.5,1112484676
3,1,32,3.5,1112484819
4,1,47,3.5,1112484727
5,1,50,3.5,1112484580
6,1,112,3.5,1094785740


In [9]:
# Count total number of unique users
@show length(unique(df_ratings[:userId]))
@show length(unique(df_ratings[:movieId]))

length(unique(df_ratings[:userId])) = 138493
length(unique(df_ratings[:movieId])) = 26744


26744

### Excercise:

In [ ]:
# Most prolific movie watcher
# Most watched movie
# Most highyl rated movie

In [12]:
# Plot of average ratings over time
# Few other plots.

### Parallel Movie recommender system 

This notebook demos collaborative filtering based movie recommender systems in Julia. The package [RecSys.jl](https://github.com/abhijithch/RecSys.jl/) is a package for recommender systems in Julia, it can currently work with explicit ratings data. This demos a parallel implementation of the ALS factorization based collaborative filtering for movie recommendations based on [this](http://dl.acm.org/citation.cfm?id=1424269) research article. The detailed report of the system is [here](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html).

### Collaborative Filtering using weighted ALS factorization

<img src="./images/als.png" width="550">

Let $U={u_i}$ be the user feature matrix where ${u_i} \subseteq\mathbb{R}^{n_f}$ and $i=1,2,...,n_u$, and let $M={m_j}$ be the item or movie feature matrix, where ${m_j} \subseteq \mathbb{R}^{n_f}$ and $j=1,2,...,n_m$. Here $n_f$ is the number of factors, i.e., the reduced dimension or the lower rank, which is determined by cross validation. The predictions can be calculated for any user-movie combination,
$(i,j)$, as $r_{ij}={u_i} \cdotp {m_j}, \forall i,j$.

** Reference ** :

[Large-Scale Parallel Collaborative Filtering for the Netflix Prize](http://dl.acm.org/citation.cfm?id=1424269)

[Movielens dataset](http://grouplens.org/datasets/movielens/)

In [ ]:
# Installation of the packages. (To be done the first time, hence comment after installing.)
#Pkg.clone("https://github.com/tanmaykm/Blobs.jl.git")
#Pkg.clone("https://github.com/abhijithch/RecSys.jl")

In [16]:
using RecSys

In [ ]:
include("/Users/abhijith/.julia/v0.5/RecSys/examples/movielens/movielens.jl")

### Dataset : 

GroupLens Research has collected and made available rating data sets from the [MovieLens](http://movielens.org) web site. The data sets were collected over various periods of time, depending on the size of the set. 

#### MovieLens 20M Dataset

Stable benchmark dataset. 20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

We use the ratings data to form a sparse matrix of size `138,000 X 27,000` with 20 million ratings ranging from 1 to 5.

In [ ]:
# Please specify path to the data folder which includes the 20 million ratings data folder "ml-20m"
dataset_path = ""

In [3]:
data_folder = "ml-20m"

"ml-20m"

Creating file handles to the movie ratings and the movies list files.

In [4]:
ratings_file = DlmFile(joinpath(dataset_path,data_folder, "ratings.csv"); dlm=',', header=true)
movies_file = DlmFile(joinpath(dataset_path,data_folder, "movies.csv"); dlm=',', header=true)

RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/movies.csv",',',true,true)

#### Parallel implementations :

This package offers 3 modes of parallelism, 

1. Multi-threading - Julia native threading infrastructure provides an easy way to make use threads.
2. Shared memory - This is a multiprocessing using shared data.
3. Distributed memory - This is distributed memory based multiprocessing, this would require that the data be split into chunks. There is code to do this, refer ...

Multiple Dispatch is a nice feature in Julia, which would dispatch to the correct implementation based on the type of the objects passed as arguments. 

For e.x., if we need to train the model using shared memory multiprocessing, the type of `MovieRec` is `MovieRec(trainingset::FileSpec, movie_names::FileSpec)` and if we need distributed memory model the type of `MovieRec` is `MovieRec(user_item_ratings::FileSpec, item_user_ratings::FileSpec, movie_names::FileSpec)`.

Let us see how Shared memory Parallel implementation trains the MovieLens data set

In [160]:
rec = MovieRec(ratings_file, movies_file, true)

MovieRec(RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/movies.csv",',',true,true),RecSys.ALSWR{RecSys.ParShmem,RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel}(RecSys.SharedMemoryInputs(RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/ratings.csv",',',true,true),0,0,Nullable{Union{ParallelSparseMatMul.SharedSparseMatrixCSC{Float64,Int64},RecSys.MatrixBlobs.SparseMatBlobs{Tv,Ti},SparseMatrixCSC{Float64,Int64}}}(),Nullable{Union{ParallelSparseMatMul.SharedSparseMatrixCSC{Float64,Int64},RecSys.MatrixBlobs.SparseMatBlobs{Tv,Ti},SparseMatrixCSC{Float64,Int64}}}(),Nullable{Union{Array{Int64,1},SharedArray{Int64,1}}}(),Nullable{Union{Array{Int64,1},SharedArray{Int64,1}}}()),Nullable{RecSys.SharedMemoryModel}(),RecSys.ParShmem()),Nullable{SparseVector{AbstractString,Int64}}())

To run in parallel mode, add processes like below:

Let us train the model with `10` factors and `10` iterations.

In [161]:
@time train(rec, 10, 10)

 94.471160 seconds (54.32 M allocations: 64.239 GB, 13.43% gc time)


In [150]:
nprocs()

1

In [162]:
err = rmse(rec)

0.7534671378202421

#### Parallel Run:

In [ ]:
# Check the number of procs
nprocs()
# Add any number of procs
#addprocs(2)
@everywhere using RecSys
@everywhere include(joinpath(Pkg.dir("RecSys"), "examples", "movielens", "movielens.jl"))

In [ ]:
@time train(rec, 10, 10)

#### Select a user, for which we show the movies watched and the recommendations for the user. 

In [33]:
user = 100
print_recommendations(rec, recommend(rec, user)...)

Already watched:
  [1 ]  =  "Nixon (1995) - Drama"
  [2 ]  =  "Leaving Las Vegas (1995) - Drama|Romance"
  [3 ]  =  "Twelve Monkeys (a.k.a. 12 Monkeys) (1995) - Mystery|Sci-Fi|Thriller"
  [4 ]  =  "Clueless (1995) - Comedy|Romance"
  [5 ]  =  "Usual Suspects, The (1995) - Crime|Mystery|Thriller"
  [6 ]  =  "From Dusk Till Dawn (1996) - Action|Comedy|Horror|Thriller"
  [7 ]  =  "Crimson Tide (1995) - Drama|Thriller|War"
  [8 ]  =  "Crumb (1994) - Documentary"
  [9 ]  =  "Net, The (1995) - Action|Crime|Thriller"
  [10]  =  "Smoke (1995) - Comedy|Drama"
  [11]  =  "Clerks (1994) - Comedy"
  [12]  =  "Ed Wood (1994) - Comedy|Drama"
  [13]  =  "Star Wars: Episode IV - A New Hope (1977) - Action|Adventure|Sci-Fi"
  [14]  =  "Like Water for Chocolate (Como agua para chocolate) (1992) - Drama|Fantasy|Romance"
  [15]  =  "Natural Born Killers (1994) - Action|Crime|Thriller"
  [16]  =  "Léon: The Professional (a.k.a. The Professional) (Léon) (1994) - Action|Crime|Drama|Thriller"
  [17]  =  "Pulp

In [18]:
;pwd

/Users/abhijith/work/training/JuliaTraining/3DayProgram/Day 1
